# NBA Elo Ratings Analysis – 21st Century Focus
With the **NBA Playoffs currently underway**, this project analyzes team performance in the 21st century using FiveThirtyEight’s `nbaallelo.csv` dataset. We limited our scope to games from 2000 onward, as the style of play, pace, and competitive dynamics have significantly changed from prior decades. The Elo rating system helps us quantify team strength over time and identify performance trends, playoff patterns, and impactful upsets.

## Questions to Explore
1. Which teams had the highest Elo rating at the end of each season since 2000?
2. How does home-court advantage impact win percentage?
3. What is the distribution of Elo ratings in the playoffs vs regular season?
4. What are the biggest Elo upsets since 2000?
5. Which franchises have the most playoff games since 2000?
6. Which teams had the largest Elo increase in a single season?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

# Load dataset
df = pd.read_csv('../data/nbaallelo.csv')
# Filter for 21st century
df = df[df['year_id'] >= 2000].copy()
df.reset_index(drop=True, inplace=True)
df.head()

### 1. Top Elo Rating at End of Each Season (2000–Present)

In [ ]:
final_games = df.groupby(['year_id', 'team_id']).apply(lambda x: x.loc[x['gameorder'].idxmax()]).reset_index(drop=True)
top_teams = final_games.loc[final_games.groupby('year_id')['elo_i'].idxmax()]
plt.figure(figsize=(12,6))
sns.lineplot(data=top_teams, x='year_id', y='elo_i', hue='team_id', marker='o')
plt.title('Top End-of-Season Elo Ratings (2000–Present)')
plt.ylabel('Elo Rating')
plt.xlabel('Season')
plt.legend(title='Team', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### 2. Home-Court Advantage Impact

In [ ]:
home_win_pct = (df[df['game_location'] == 'H']['game_result'] == 'W').mean()
away_win_pct = (df[df['game_location'] == 'A']['game_result'] == 'W').mean()
plt.bar(['Home', 'Away'], [home_win_pct, away_win_pct], color=['blue', 'orange'])
plt.title('Win Percentage: Home vs Away')
plt.ylabel('Win %')
plt.show()
print(f'Home win %: {home_win_pct:.2%}, Away win %: {away_win_pct:.2%}')

### 3. Elo Rating Distribution – Regular Season vs Playoffs

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(data=df, x='is_playoffs', y='elo_i')
plt.title('Elo Rating Distribution by Game Type')
plt.xlabel('0 = Regular Season, 1 = Playoffs')
plt.ylabel('Elo Rating')
plt.show()

### 4. Biggest Elo Upsets (2000–Present)

In [ ]:
df['elo_diff'] = df['opp_elo_i'] - df['elo_i']
upsets = df[(df['game_result'] == 'W') & (df['elo_diff'] > 0)].sort_values(by='elo_diff', ascending=False)
upsets[['date_game', 'team_id', 'opp_id', 'elo_i', 'opp_elo_i', 'elo_diff']].head()

### 5. Most Playoff Games by Franchise (2000–Present)

In [ ]:
playoff_counts = df[df['is_playoffs'] == 1]['fran_id'].value_counts().head(10)
sns.barplot(x=playoff_counts.values, y=playoff_counts.index, palette='viridis')
plt.title('Top 10 Franchises by Playoff Games Played')
plt.xlabel('Playoff Games')
plt.ylabel('Franchise')
plt.show()

### 6. Largest Elo Increase in a Single Season

In [ ]:
season_elo = df.groupby(['year_id', 'team_id'])['elo_i'].agg(['first', 'last'])
season_elo['elo_gain'] = season_elo['last'] - season_elo['first']
top_gainers = season_elo.sort_values(by='elo_gain', ascending=False).head(10).reset_index()
sns.barplot(data=top_gainers, x='elo_gain', y='team_id', hue='year_id', dodge=False)
plt.title('Top 10 Team-Season Elo Gains')
plt.xlabel('Elo Rating Gain')
plt.ylabel('Team')
plt.show()

## Summary & Insights
- Top end-of-season Elo ratings highlight the most dominant teams year by year.
- Home teams continue to win more often, though not overwhelmingly.
- Playoff Elo ratings are generally higher, showing the intensity of postseason matchups.
- Notable upsets exist where teams overcame significant Elo disadvantages.
- Some franchises are consistent playoff contenders, while others fluctuate.
- Elo gains across seasons reveal which teams made the biggest improvements in single-year spans.

This analysis helps illustrate modern team performance using objective rating metrics like Elo.